# Pivot table

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

d:\Users\Renan Lemes\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [4]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [5]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [6]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [7]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

In [8]:
df_pivot = titanic.pivot_table('survived', ['sex', age], ['who', 'class'])
df_pivot

who                 child                     man                      \
class               First Second     Third  First    Second     Third   
sex    age                                                              
female (0, 18]   0.666667    1.0  0.533333    NaN       NaN       NaN   
       (18, 80]       NaN    NaN       NaN    NaN       NaN       NaN   
male   (0, 18]   1.000000    1.0  0.321429  0.500  0.000000  0.086957   
       (18, 80]       NaN    NaN       NaN  0.375  0.071429  0.133663   

who                 woman                   
class               First Second     Third  
sex    age                                  
female (0, 18]   1.000000    1.0  0.461538  
       (18, 80]  0.972973    0.9  0.423729  
male   (0, 18]        NaN    NaN       NaN  
       (18, 80]       NaN    NaN       NaN

In [9]:
df_pivot['child', 'First']

sex     age     
female  (0, 18]     0.666667
        (18, 80]         NaN
male    (0, 18]     1.000000
        (18, 80]         NaN
Name: (child, First), dtype: float64

In [10]:
df_pivot['child']

class               First  Second     Third
sex    age                                 
female (0, 18]   0.666667     1.0  0.533333
       (18, 80]       NaN     NaN       NaN
male   (0, 18]   1.000000     1.0  0.321429
       (18, 80]       NaN     NaN       NaN

In [11]:
titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

# Wide to long

Também conhecido como melt ou unmelt

In [15]:
df = pd.DataFrame({'col_1': ['A'] * 3 + ['B']*3,
                   'col_2': ['Primeiro', 'Segundo', 'Terceiro'] * 2,
                   'col_3': [1, 2, 3, 4, 5, 6]})

In [16]:
df_wide = df.pivot(index='col_1', columns= 'col_2', values='col_3')
df_wide

col_2,Primeiro,Segundo,Terceiro
col_1,,,
A,1,2,3
B,4,5,6


In [17]:
df_long = df_wide.reset_index().melt(id_vars='col_1', value_vars=['Primeiro', 'Segundo', 'Terceiro'])
df_long

,col_1,col_2,value
0,A,Primeiro,1
1,B,Primeiro,4
2,A,Segundo,2
3,B,Segundo,5
4,A,Terceiro,3
5,B,Terceiro,6


In [18]:
df_wide = pd.DataFrame({'col_1': ['A', 'B'],
                        'Primeiro': [1, 4],
                        'Segundo': [2, 5],
                        'Terceiro': [3, 6]})

df_wide.melt(id_vars='col_1', value_vars=['Primeiro', 'Segundo', 'Terceiro'])

,col_1,variable,value
0,A,Primeiro,1
1,B,Primeiro,4
2,A,Segundo,2
3,B,Segundo,5
4,A,Terceiro,3
5,B,Terceiro,6


In [19]:
df_wide.melt(id_vars='col_1', value_vars=['Primeiro', 'Segundo', 'Terceiro'], ignore_index=False)


,col_1,variable,value
0,A,Primeiro,1
1,B,Primeiro,4
0,A,Segundo,2
1,B,Segundo,5
0,A,Terceiro,3
1,B,Terceiro,6
